<a href="https://colab.research.google.com/github/Eandrushenko/CPTS-437/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elijah Andrushenko
03-05-2018
CPTS 437
Introduction to Machine Learning
Homework # 4
---
1.
Error done with calculator by hand:
My calculations:
•Take Y columns and find the average. Average was 126.7.
•Error equals each value in Y minus the Average.
•Square each value that was subtracted from the average.
•Find the average of the those values.
•The Error squared was 4766.7065

New Theta0 = 7.42979
New Theta1 = -0.00284

Thetas were calculated by the first code block below, code for this part was found by a link referenced in that code block.

2.
https://prnt.sc/mu3l24
https://faculty.washington.edu/heagerty/Books/Biostatistics/TABLES/t-Tables/

3.
**•The bigger the  λ**
-The smaller the w
-The larger the variance
-Not so great with high complexity datasets

**•The smaller the λ**
-The bigger the w
-The larger the bias
-Typical to overfit

4.
•The project title
      Binary Planet Classification
•The machine learning methods you propose to element
      The goal of our project is to be able to classify a planet as either life supporting or non-life supporting. We think that if we can get enough data, with the correct features then we should be able to use a trained K-N-N model to identify new planets that our classifier hasn't seen as either possibility for life or no possibility for life
•The dataset(s) you will use to evaluate your methods
  We have found two data sets on kaggle that we can use to develop our project, Exoplantes Database and Open Exoplanet Catalogue. Since these data sets are not labeled, We are going to have to find a way of providing labels because K-N-N is a supervised learning model. I think we can get around this by using K-means clustering to provide us with a list of clusters which we can then use as our labels for the K-N-N algorithm.
•The performance objective and measures you will use
    Since scientists have been able to identify some planets that may actually have a possibility for life we can use these planets as our test values. However one problem is that we will have very few test values so we may have to employ some techniques that we have learned about in class to boost the number of points in a data set.
•The names and roles of every person on your project team
  Jake Gergen , Elijah Andrushenko
•What you hope to learn from the project and how the results could be used by experts
  We hope to be able to accuratly predict which planets have a possbility for life and which ones don't, as well as hopfully be able to discover planets that actually do have a possiblity for life that are not yet known.
 •Milestones:
 3/7 - Get project approved, Get a good data set, if not multiple
 4/11 - Implementation of machine learning algorithms, verify correct work, graphed results, other visual data represented, begin making other tests, and begin display for professor.
 Demonstration Data - All work should be complete a day before demonstration date if not sooner.
 

5.
Average Accuracies:
-Gaussian NB: 85.24
-10-fold Binning: 77.05
-One VS Rest Accuracy, coded: 72.26
-One VS Rest Accuracy, sklearn: 80.22



In [0]:
#https://towardsdatascience.com/linear-regression-simplified-ordinary-least-square-vs-gradient-descent-48145de2cf76

X = [0.07786339, -0.03961813, 0.01103904, -0.04069594, -0.03422907, 0.00564998, 0.08864151, -0.03315126, -0.05686312, -0.03099563, 
     0.05522933, -0.06009656, 0.00133873, -0.02345095, -0.07410811, 0.01966154, -0.01590626, -0.01590626, 0.03906215, -0.07303030]
Y = [233,	91, 111, 152, 120, 67, 310, 94, 183, 66, 173, 72, 49, 64, 48, 178, 104, 132, 220, 67]

t0 = 7.42968
t1 = -0.002945
a = 0.0001

def gradientDescent(X, y, theta, alpha, num_iters):
    """
       Performs gradient descent to learn theta
    """
    m = len(y)  # number of training examples
    for i in range(num_iters):
        y_hat = np.dot(X, theta)
        theta = theta - alpha * (1.0/m) * np.dot(X, y_hat-y)
    return theta

updated0 = gradientDescent(X, Y, t0, a, 1)
updated1 = gradientDescent(X, Y, t1, a, 1)

print(updated0)
print(updated1)


In [0]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from google.colab import drive
from google.colab import files

def read_data():
  uploaded = files.upload()
  data = np.loadtxt(fname='smarthome.csv', delimiter=',')
  X = data[:,:-1]
  Y = data[:,-1]
  return X,Y

X, Y = read_data()

In [0]:
classlist = np.unique(Y)

def build(X,Y):
    clf = GaussianNB()
    scores = cross_val_score(clf, X, Y, cv = 10)
    scores = scores * 100
    return scores

#Gaussian Naive Bayes
accuracy = build(X,Y)
print ("Gaussian NB: " + str(numpy.mean(accuracy)))  
  
def discretized(X):
  max_features = np.max(X, axis = 0)
  row = X.shape[0]
  column = X.shape[1]
  RA = np.zeros((10,column))
  dis = numpy.zeros((row,column), numpy.int8)
  for features in range(column): 
    cur = 0 
    step = max_features[features] / 10.00 
    RA[9][features] = max_features[features]
    for i in range(9):
      cur = cur + step
      RA[i][features] = cur
  for features in range (column):
    for i in range(row):
      val = X[i][features]
      for binning in range(10):
        if val > RA[binning][features] :
          continue 
        else:
          dis[i][features] = int(binning)
          break
  return dis 

#Binning
binning = discretized(X)
accuracy = build(binning,Y)
print("10-fold Binning: " + str(np.mean(accuracy)))

def train(X, Y, classes):
    instances = X.shape[0]
    gaussian = []
    for c in classes:
        binary = numpy.copy(Y)
        for i in range(instances):
            label = Y[i]
            if label == c:
                binary[i] = 1
            else:
                binary[i] = -1
        gauss = GaussianNB()
        gauss.fit(X, binary)
        gaussian.append(gauss)
            
    return gaussian
  
def test(clf, X, Y, classes):
    score = [0] * len(classes)
    instances = X.shape[0] 
    predictions = np.zeros((Y.shape[0],1), numpy.int8)
    for i in range(instances):
        current = X[i]
        score = [0] * len(classes)
        for c in range(len(classes)):
            y = clf[c].predict([current])
            score[c] = score[c] + y
        predictions[i] = np.argmax(score)
    correct = 0
    for i in range(instances):
        if Y[i] == predictions[i]:
            correct += 1
    return (float(correct) / float(instances))*100

#One vs Rest without Sklearn
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
classifiers = train(X_train, Y_train, classlist)
accuracy = test(classifiers, X_test, Y_test, classlist)
print("One VS Rest Accuracy: "  +  str(accuracy))

#One vs Rest with Sklearn
classifiers = OneVsRestClassifier(GaussianNB())
classifiers.fit(X_train, Y_train)
accuracy  = classif.score(X_test, Y_test) * 100     
print ("One VS Rest Accuracy: "  +  str(accuracy))

In [0]:
class K_Means:

	def __init__(self, k =3, tolerance = 0.0001, max_iterations = 500):
		self.k = k
		self.tolerance = tolerance
		self.max_iterations = max_iterations